<a href="https://colab.research.google.com/github/AmedeeRoy/BirdDL/blob/main/projection/LB_unet_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive/")

%cd drive/My\ Drive/


Mounted at /content/drive/
/content/drive/My Drive


# LOAD MODELS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import sqlite3

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from utils.trip import *
from utils.unet import *

# Load data

In [3]:
data_all = pd.read_table('./data/gps+5-30s.csv', sep = ',')
check = pd.read_table('./data/check_gps+5-30s.csv', sep = ',')

# LB 5s
## data

In [4]:
select = check.trip[(check.species == 'LB') & (check.res == 5)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
678346,P1108_55_LB_T1,2008-11-25 13:47:56,-77.275767,-11.758682,0.0,9.973368,-38.030537,0,2.054578,-1.057361,0.538182,0.787683,-0.616081
678347,P1108_55_LB_T1,2008-11-25 13:48:01,-77.276120,-11.758358,0.0,10.541805,3.302871,0,2.050876,-1.044847,0.569015,0.998339,0.057614
678348,P1108_55_LB_T1,2008-11-25 13:48:06,-77.276408,-11.757928,0.0,11.450902,13.643503,0,2.047856,-1.028208,0.618325,0.971782,0.235880
678349,P1108_55_LB_T1,2008-11-25 13:48:11,-77.276813,-11.757527,0.0,12.563934,-11.353300,0,2.043612,-1.012667,0.678698,0.980432,-0.196858
678350,P1108_55_LB_T1,2008-11-25 13:48:16,-77.277228,-11.757092,0.0,13.251292,1.588349,0,2.039264,-0.995834,0.715981,0.999616,0.027718
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684250,P1113_9_LB_T2,2013-11-15 16:37:20,-77.266136,-11.774673,0.0,13.387964,20.986559,0,2.204501,1.415908,0.326915,0.933664,0.358149
1684251,P1113_9_LB_T2,2013-11-15 16:37:25,-77.265480,-11.774356,0.0,15.943802,-57.190877,0,2.225848,1.426970,0.389325,0.541842,-0.840480
1684252,P1113_9_LB_T2,2013-11-15 16:37:30,-77.264725,-11.774122,0.0,17.259453,8.704263,0,2.250416,1.435135,0.421452,0.988483,0.151334
1684253,P1113_9_LB_T2,2013-11-15 16:37:35,-77.264069,-11.774305,0.0,14.866032,33.472913,0,2.271763,1.428749,0.363008,0.834147,0.551543


In [5]:
model = torch.load('LB_unet_matrixlonlatcov_5s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [6]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

P1108_55_LB_T1
P1108_55_LB_T2
P1109_15_LB_T1
P1109_15_LB_T2
P1109_15_LB_T3
P1109_15_LB_T4
P1109_19_LB_T1
P1109_19_LB_T2
P1109_19_LB_T3
P1109_19_LB_T4
P1109_19_LB_T5
P1109_19_LB_T6
P1109_2_LB_T1
P1109_2_LB_T2
P1109_2_LB_T3
P1110_45_LB_T1
P1110_45_LB_T2
P1110_45_LB_T3
P1110_45_LB_T4
P1110_65_LB_T1
P1110_65_LB_T2
P1110_65_LB_T3
P1110_65_LB_T4
P1110_65_LB_T5
P1110_65_LB_T6
P1110_66_LB_T1
P1110_66_LB_T2
P1110_66_LB_T3
P1110_66_LB_T4
P1110_69_LB_T1
P1110_69_LB_T2
P1110_69_LB_T3
P1110_69_LB_T4
P1110_69_LB_T5
P1110_69_LB_T6
P1110_77_LB_T1
P1110_77_LB_T2
P1110_77_LB_T3
P1110_77_LB_T4
P1111_1_LB_T1
P1111_1_LB_T2
P1111_14_LB_T1
P1111_14_LB_T2
P1111_14_LB_T3
P1111_14_LB_T4
P1111_15_LB_T1
P1111_15_LB_T2
P1111_15_LB_T3
P1111_15_LB_T4
P1111_15_LB_T5
P1111_16_LB_T1
P1111_16_LB_T2
P1111_16_LB_T3
P1111_18_LB_T1
P1111_18_LB_T2
P1111_18_LB_T3
P1111_18_LB_T4
P1111_19_LB_T1
P1111_19_LB_T2
P1111_19_LB_T3
P1111_19_LB_T4
P1111_21_LB_T1
P1111_21_LB_T2
P1111_21_LB_T3
P1111_22_LB_T1
P1111_22_LB_T2
P1111_22_LB_T3


In [7]:
data.to_csv('LB_5s.csv', index = False)

# LB 15s
## data

In [8]:
select = check.trip[(check.species == 'LB') & (check.res == 15)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
752451,P1110_12_LB_T1,2010-11-23 12:28:22,-77.270782,-11.763110,0.0,13.228726,6.657335,0,1.648771,-2.391422,0.773231,0.993257,0.115931
752452,P1110_12_LB_T1,2010-11-23 12:28:37,-77.271108,-11.761366,0.0,13.152918,4.618355,0,1.634926,-2.366896,0.768790,0.996753,0.080518
752453,P1110_12_LB_T1,2010-11-23 12:28:52,-77.271510,-11.759487,0.0,14.246502,-1.470654,0,1.617829,-2.340464,0.832857,0.999671,-0.025665
752454,P1110_12_LB_T1,2010-11-23 12:29:07,-77.272280,-11.757767,0.0,13.940894,-11.840893,0,1.585066,-2.316261,0.814953,0.978721,-0.205195
752455,P1110_12_LB_T1,2010-11-23 12:29:22,-77.273010,-11.756070,0.0,13.665175,0.837671,0,1.554026,-2.292387,0.798800,0.999893,0.014620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142223,P1111_12_LB_T2,2011-11-14 20:52:26,-77.265092,-11.776497,0.0,8.953878,47.056350,0,2.446241,0.871704,0.432013,0.681279,0.732024
1142224,P1111_12_LB_T2,2011-11-14 20:52:41,-77.266487,-11.775593,0.0,12.152338,51.147458,0,2.424418,0.897138,0.586334,0.627318,0.778763
1142225,P1111_12_LB_T2,2011-11-14 20:52:56,-77.264670,-11.774316,0.0,16.248001,110.830302,0,2.452840,0.933073,0.783945,-0.355601,0.934638
1142226,P1111_12_LB_T2,2011-11-14 20:53:11,-77.264856,-11.775627,0.0,9.818410,133.584359,0,2.449931,0.896192,0.473725,-0.689422,0.724360


In [9]:
model = torch.load('LB_unet_matrixlonlatcov_15s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [10]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

P1110_12_LB_T1
P1110_12_LB_T2
P1110_12_LB_T3
P1110_12_LB_T4
P1110_13_LB_T1
P1110_13_LB_T2
P1110_25_LB_T1
P1110_25_LB_T2
P1110_25_LB_T3
P1110_35_LB_T1
P1110_35_LB_T2
P1110_35_LB_T3
P1110_55_LB_T1
P1110_55_LB_T2
P1111_11_LB_T1
P1111_11_LB_T2
P1111_12_LB_T1
P1111_12_LB_T2


In [11]:
data.to_csv('LB_15s.csv', index = False)

# LB 30s
## data

In [12]:
select = check.trip[(check.species == 'LB') & (check.res == 30)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
676794,P1108_52_LB_T1,2008-11-23 12:53:25,-77.277057,-11.765532,0.0,13.639678,17.605840,0,0.172864,-1.729367,0.920700,0.953160,0.302467
676795,P1108_52_LB_T1,2008-11-23 12:53:55,-77.278930,-11.763129,0.0,11.217172,-5.918671,0,-0.091171,-0.757726,0.756072,0.994669,-0.103117
676796,P1108_52_LB_T1,2008-11-23 12:54:25,-77.280679,-11.762089,0.0,7.432729,-21.386366,0,-0.337670,-0.337413,0.498890,0.931143,-0.364655
676797,P1108_52_LB_T1,2008-11-23 12:54:55,-77.280733,-11.762051,0.0,0.240742,4.994635,0,-0.345201,-0.321898,0.010139,0.996203,0.087062
676798,P1108_52_LB_T1,2008-11-23 12:55:25,-77.280827,-11.762030,0.0,0.351722,-23.725055,0,-0.358522,-0.313582,0.017681,0.915487,-0.402348
...,...,...,...,...,...,...,...,...,...,...,...,...,...
950677,P1110_55_LB_T3,2010-12-04 16:53:19,-77.261584,-11.770655,0.0,10.058552,2.481198,0,-2.799633,2.791577,0.320209,0.999062,0.043292
950678,P1110_55_LB_T3,2010-12-04 16:53:49,-77.262591,-11.772643,0.0,8.234080,-161.438015,0,-2.856395,2.673539,0.262128,-0.947980,-0.318330
950679,P1110_55_LB_T3,2010-12-04 16:54:19,-77.264258,-11.774622,0.0,9.517505,13.154879,0,-2.950401,2.556077,0.302985,0.973758,0.227584
950680,P1110_55_LB_T3,2010-12-04 16:54:49,-77.264056,-11.773545,0.0,4.062153,150.892711,0,-2.939000,2.620001,0.129317,-0.873710,0.486447


In [13]:
model = torch.load('LB_unet_matrixlonlatcov_30s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [14]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

P1108_52_LB_T1
P1108_52_LB_T2
P1108_52_LB_T3
P1108_53_LB_T1
P1108_53_LB_T2
P1108_53_LB_T3
P1108_54_LB_T1
P1108_54_LB_T2
P1108_54_LB_T3
P1108_56_LB_T1
P1108_56_LB_T2
P1108_56_LB_T3
P1108_58_LB_T1
P1108_58_LB_T2
P1108_58_LB_T3
P1108_59_LB_T1
P1108_59_LB_T2
P1108_60_LB_T1
P1108_60_LB_T2
P1108_60_LB_T3
P1108_61_LB_T1
P1108_61_LB_T2
P1108_61_LB_T3
P1108_61_LB_T4
P1108_62_LB_T1
P1108_62_LB_T2
P1108_62_LB_T3
P1108_62_LB_T4
P1108_63_LB_T1
P1108_63_LB_T2
P1108_63_LB_T3
P1108_64_LB_T1
P1108_64_LB_T2
P1108_64_LB_T3
P1108_66_LB_T1
P1108_67_LB_T1
P1108_67_LB_T2
P1108_67_LB_T3
P1108_68_LB_T1
P1108_68_LB_T2
P1110_55_LB_T3


In [15]:
data.to_csv('LB_30s.csv', index = False)